Author: Simon Ng <br/> 
Created: 2024-12-19 <br/> 
Last edited: 2025-02-11 <br/><br/>
This code is the 4th step in a project to 3D print a model of the Engadin valley in Switzerland as a Christmas gift to a friend who is skiing the Engadin Ski Marathon this year! The project has 5 steps: <br/>
1. Select area of interest from an open source Swiss elevation dataset (https://www.swisstopo.admin.ch/en/height-model-swissalti3d). I used "selection by polygon" at 2.0m resolution to obtain a csv file with links to tiles covering the entire Engadin Ski Marathon and surrounding mountains. <br/>
2. Merge linked GeoTiff tiles in exported csv file into one file <br/>
3. Add geospatial features in QGIS, ArcGIS, etc such as trails, cities, etc. Note that additional features need to be represented as elevation data merged with the exported GeoTiff so they will be part of the eventual 3D print! Also a good time to resample the pixel resolution. 2 meter resolution won't show up for a reasonably scaled 3D print but WILL make processing times horrific. Something like 30 meter resolution is better. This is also the time to re-mask your area of interest since the swisstopo site exports blocky edges on diagonals. <br/>
4. Convert edited GeoTiff to STL mesh for slicing and 3D printing (<<< that's what this code does!) 
5. Slice and 3D print it!

In [1]:
# Import packages
import xarray
import rioxarray # to open and download remote raster data
from rioxarray.merge import merge_arrays
import numpy as np
from stl import mesh

C:\Users\ngsim\UserApps_SN\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# Load edited GeoTiff from step 3
tif_path = r"C:\Users\ngsim\Documents\Engadin\241218_EngadinValley_30m_QGIS_processed.tif" # GeoTiff exported from step 3
MergedTiles = rioxarray.open_rasterio(tif_path) # open tif as xarray DataArray
MergedTiles.shape # print DataArray shape

C:\Users\ngsim\UserApps_SN\Anaconda3\lib\site-packages\pyproj\crs\_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


(1, 1267, 1367)

Process GeoTiff image to be ready for STL mesh conversion

In [3]:
# Convert xarray DataArray into numpy array for easier processing (don't need CRS anymore, just elevation pixel values)
tif_array = MergedTiles.values[0] # -GPT  # make tif_array 2D since 3rd dimension isn't useful. ChatGPT failed to recognize the simplification of removing the 3rd unecessary dimension

# Adjust base elevation value
base_elevation = 800  # this value (meters) will determine how tall the 3D print is. (i.e. if the minimum elevation in the study area is 1500m, then a base elevation of 1000m would give 500m of 3D print (scaled to the print size of course) beneath that minimum elevation)
tif_array_wBase = np.where(tif_array > base_elevation, tif_array, base_elevation)  # Replace excluded values with lowest desired model elevation
print(f"Base elevation is now set to {round(tif_array_wBase.min())} meters, with a maximum elevation of {round(tif_array_wBase.max())} meters.")

# Scale elevation z-data by pixel x-y dimensions
pixel_size = 30 # tif pixel size in meters (this should match the pixel size in meters as exported in step 3)
elevation_scale_factor = 2 # if you want to make the mountains look bigger than they really are for effect
height_map = tif_array_wBase/pixel_size*elevation_scale_factor # scale height by pixel size (in meters)
rows, cols = height_map.shape # -GPT Get the dimensions of the image
print(f"With a pixel size of {pixel_size} and elevation scale factor of {elevation_scale_factor}, the newly scaled minimum height value is {round(height_map.min())} and maximum height value is {round(height_map.max())} (now unitless).")

Base elevation is now set to 800 meters, with a maximum elevation of 3444 meters.
With a pixel size of 30 and elevation scale factor of 2, the newly scaled minimum height value is 53 and maximum height value is 230 (now unitless).


In [4]:
# Optional code to split map into 5 vertical pieces to reduce memory requirements

split_map = False

if split_map:
    row_segments = round(rows/5) # number of rows in each map segment
    map_segment1 = height_map[:row_segments] # slice out first set of rows
    map_segment2 = height_map[row_segments:row_segments*2] # slice out second set of rows, etc
    map_segment3 = height_map[row_segments*2:row_segments*3]
    map_segment4 = height_map[row_segments*3:row_segments*4]
    map_segment5 = height_map[row_segments*4:]
    rows = row_segments # update rows for subsequent vertices and faces loops to run through each map segment instead of the entire map
    height_map = map_segment1 # udpate height_map for subsequent loops with desired map segment to process
    print(f"Map segments rows, columns = {map_segment1.shape}") # print the map segment rows and columns

Convert processed tif into stl mesh, including creating a mesh to calculate only valid elevation datapoints

In [5]:
def add_mask_edges(mask):
        '''
        add_mask_edges background explanation: 
        The "mask" crops the elevation data to only include real elevation data, getting rid of the extra pixels on each side arising 
        from the not perfect north-south-east-west orientation of the area of interest.
        
        To goal of this function is to add back one pixel on each edge of the real data so that the eventual stl mesh will include "sides" for 3D printing rather than just being the top elevation surface
        The code was inspired and structured by ChatGPT, in particular the idea to use np.pad and then look through each pixel to construct a mask including edges.
        However, ChatGPT's 2 attempts were difficult for me to understand and did not work as intended.
        So, I modified ChatGPT's concept with my own method. I've attempted to mark where ChatGPT's code and comment contributions are with "-GPT". If there is a comment after # -GPT, it is my own.
            
         Parameters:
            mask (numpy.ndarray): A boolean mask where True indicates valid pixels and False indicates excluded pixels. -GPT edited
    
         Returns:
            numpy.ndarray: A boolean mask where True indicates valid pixels and pixels adjacent to valid pixels. -GPT edited'''
        
        print("Creating padded mask to generate sides for STL file...")
        
        # Create a padded version of the mask to handle edge cases -GPT
        padded_mask = np.pad(mask, pad_width=1, mode='constant', constant_values=False) #  -GPT # pads each dimension of the array with 1 row/column. i.e. (380, 594) becomes (382, 596)
        #print(f"Padded mask shape is {padded_mask.shape}")
        
        # Initialize the new mask  -GPT
        mask_plus_edges = np.zeros_like(mask, dtype=bool) # -GPT # has same size as original mask, all set to false 
        
        # Check each pixel in original mask
        for i in range(mask.shape[0]): # for rows
            for j in range(mask.shape[1]): # and columns
                
                # Check all 8 surrounding pixels (and self-pixel) to see if the original mask includes any one of the pixels (i.e. =True)
                for i_offset in [-1, 0, 1]: # -GPT   # look in rows
                    for j_offset in [-1, 0, 1]: # -GPT   # and in columns
                        
                        # Get row and column indices for padded mask
                        row = i + i_offset + 1 # +1 is required to account for the extra left/top column/row in the padded array
                        col = j + j_offset + 1
                        
                        if padded_mask[row][col]: # if adjacent or central cell is true in mask (padded mask allows original mask edges to be evaluated without error)
                            mask_plus_edges[i][j] = True # then central cell should be included in the new mask
        
        print("Padded mask complete")   
        
        return mask_plus_edges

In [6]:
# Create mask to only process pixels of real elevation data
exclude_value = tif_array.min() # -GPT # excluding the minimum value removes areas outside the desired mountainous region that are either assigned 0 or -9999 depending on how the tif was exported
print(f"'no-elevation-data' value is {exclude_value}")
mask_woEdges = tif_array != exclude_value  # -GPT # Create mask that is True where pixels have real elevation data
mask = add_mask_edges(mask_woEdges) # add one column/row of True values around the original mask so that "sides" will be calculated for the stl file
#tif_array *= mask  # -GPT   Apply mask to keep only valid regions. Applying the mask here doesn't actually do anything...

'no-elevation-data' value is 0.0
Creating padded mask to generate sides for STL file...
Padded mask complete


In [7]:
# Generate mesh and export stl file (code mostly from ChatGPT, though of course I had to update prompts to get a code that would do exactly what I wanted)
# I made edits where noted (-Simon edit) to improve processing time (changing vartices and faces arrays to int). Processing time was an issue before I reduced the pixel resolution. With the current 30m pixels, it doesn't really matter if the vertices and faces are stored as int or float

# Create vertices (only for valid pixels)
print("Creating vertices...")  # -Simon edit

vertices = []
vertex_map = -np.ones((rows, cols), dtype=int)  # Map to track valid vertices
vertex_index = 0

for i in range(rows):
    for j in range(cols):
        if mask[i, j]:  # Only add vertices for valid pixels
            vertices.append([i, j, height_map[i, j]])
            vertex_map[i, j] = vertex_index
            vertex_index += 1

vertices = np.array(vertices).astype(int) # -Simon edit. Convert vertices list to numpy array as integer to reduce processing time


# Create faces (only for valid grid cells)
print("Creating faces...")  # -Simon edit

faces = []
for i in range(rows - 1):
    for j in range(cols - 1):
        if mask[i, j] and mask[i, j + 1] and mask[i + 1, j]:  # Valid first triangle
            top_left = vertex_map[i, j]
            top_right = vertex_map[i, j + 1]
            bottom_left = vertex_map[i + 1, j]
            faces.append([top_left, bottom_left, top_right])

        if mask[i + 1, j] and mask[i, j + 1] and mask[i + 1, j + 1]:  # Valid second triangle
            top_right = vertex_map[i, j + 1]
            bottom_left = vertex_map[i + 1, j]
            bottom_right = vertex_map[i + 1, j + 1]
            faces.append([top_right, bottom_left, bottom_right])

faces = np.array(faces).astype(int) # -Simon edit. Convert faces list to numpy array as integer to reduce processing time

# Create the mesh
print("Creating the mesh...")  # -Simon edit

terrain_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))

for i, face in enumerate(faces):
    for j in range(3):
        terrain_mesh.vectors[i][j] = vertices[face[j], :]
        
# Save as STL
stl_path = r"C:\Users\ngsim\Documents\Engadin\250211_EngadinValley_30m.stl"
terrain_mesh.save(stl_path)
print(f"STL saved to {stl_path}")

Creating vertices...
Creating faces...
Creating the mesh...
STL saved to C:\Users\ngsim\Documents\Engadin\250211_EngadinValley_30m.stl
